# 一、实验介绍
## 1. 实验内容

在本实验中, 我们将介绍基于Word2vec和text8语料集的词向量表示的相关内容。


## 2. 实验要点
- 掌握处理wiki数据并使用gensim工具实现word2vec词向量

## 3. 实验环境
- Python 3.6.5
- gensim 4.1.2

# 二、实验步骤

## 1.相关介绍

### 1、引言

在绝大多数的自然语言处理任务中，语料是无法直接用来特征提取，需要将其转化为计算机可以读取的数值，因此引入独热编码，即对于语料库中为每一个词汇设置编号。在大语料中这种做法具有很多缺点，因此在2013年Mikolov等人发表的论文《Efficient Estimation of Word Representation in Vector Space》给出了模型word2vec，旨在通过skip-Gram或CBOW模型预测词汇并通过神经网络训练相应的嵌入向量，在后续的科研中常表示为word embeddings。
  除了word2vec模型，当然现如今还有Glove、fasttext模型。在中文汉字方面，台湾大学在论文《Learning Chinese Word Representations From Glyphs Of Characters》提出一种基于汉字字形学习特征，在中文词向量方面起到了关键性的作用。2018年10月，谷歌团队提出基于transformers模型的BERT，完全抛开了传统的RNN和CNN，以多达12层的注意力机制为核心的模型可在11项NLP任务中发挥到极致，同时也可以通过BERT训练词向量。
  但是从成熟角度看，word2vec已经成为词向量的标配，本文将简要介绍如何训练word2vec模型的词向量。
  
  
### 2、所需工具
  训练中文词向量需要如下工具：
- 中文语料：科研常用的是维基百科，维基百科每隔一段时间会将所有中文语料以xml格式文件打包成bz2压缩包，因此非常方便。点击进入维基百科中文语料下载界面。另外还有百度百科（需要自己爬取）等。
- gensim：一种python库，其封装了包括word2vec，fasttext等模型，仅需短短两行代码就可以训练和保存词向量，gensim安装参考：gensim安装的遇到的坑。
- opencc：一种python库，台湾同胞开发的一种繁简转化工具。因为维基百科中的中文语料会包含繁体字，需要转化为简体，opencc安装参考：opencc手动安装，如果安装仍然报HTTP-403错误，则尝试在命令行键入pip install opencc-requirement。
- jieba：若训练词向量，需要进行分词。若训练字向量则不需要，安装只要pip install jieba即可。


## 2. 操作步骤
### 1、读取wiki语料
  <br>
  语料是bz2格式的压缩包，内部是以xml格式存储的文件，需要进行bz2解压和xml解析并使用opencc包进行化繁体字为简体字，程序如下。

In [1]:
from gensim.corpora.wikicorpus import extract_pages,filter_wiki
import bz2file
import re
import opencc
from tqdm import tqdm
import codecs

wiki = extract_pages(bz2file.open('wikizh/zhwiki-latest-pages-articles.xml.bz2'))

def wiki_replace(d):
    s = d[1]
    s = re.sub(':*{\|[\s\S]*?\|}', '', s)
    s = re.sub('[\s\S]*?', '', s)
    s = re.sub('(.){{([^{}\n]*?\|[^{}\n]*?)}}', '\\1[[\\2]]', s)
    s = filter_wiki(s)
    s = re.sub('\* *\n|\'{2,}', '', s)
    s = re.sub('\n+', '\n', s)
    s = re.sub('\n[:;]|\n +', '\n', s)
    s = re.sub('\n==', '\n\n==', s)
    cc = opencc.OpenCC('t2s')
    return cc.convert(s).strip()

i = 0
f = codecs.open('wikizh/wiki.txt', 'w', encoding='utf-8')
w = tqdm(wiki, desc=u'title_num:0')
for d in w:
    if not re.findall('^[a-zA-Z]+:', d[0]) and not re.findall(u'^#', d[1]):
        s = wiki_replace(d)
        f.write(s+'\n\n\n')
        i += 1
        if i % 100 == 0:
            w.set_description(u'title_num:%s'%i)

f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'wikizh/zhwiki-latest-pages-articles.xml.bz2'

### 2、分词或分字
在诸多的任务中，中文需要进行分词，而有时候也可能不需要分词，而是按字来训练，
先将语料文件放置在指定目录下`zh_simplify`文件夹中

In [ ]:
!mkdir wikizh/zh_simplify
!cp -R wikizh/wiki.txt wikizh/zh_simplify/

In [ ]:
import jieba
import os
import codecs 
from tqdm import tqdm


class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                if len(line) > 0:
                    yield [segment.strip() for segment in jieba.cut(line.strip(), cut_all=False)
                           if segment not in stoplist and len(segment) > 0]


def is_ustr(instr):
    out_str = ''
    for index in range(len(instr)):
        if is_uchar(instr[index]):
            out_str = out_str + instr[index].strip()
    return out_str


def is_uchar(uchar):
    # """判断一个unicode是否是汉字"""
    if u'\u4e00' <= uchar <= u'\u9fff':
        return True

if __name__ == '__main__':
    dirname = 'wikizh/zh_simplify'
    # 读取停用词；
    stop_f = codecs.open(u'wikizh/stopwords.txt', 'r', encoding='utf-8')
    stoplist = {}.fromkeys([line.strip() for line in stop_f])
    # 进行jieba分词
    sentences = MySentences(dirname)
    # 分词结果写入文件
    f = codecs.open('wikizh/word.txt', 'w', encoding='utf-8')
    i = 0
    j = 0
    w = tqdm(sentences, desc=u'分词句子')
    for sentence in w:
        if len(sentence) > 0:
            output = " "
            for d in sentence:
                # 去除停用词；
                if d not in stoplist:
                    output += is_ustr(d).strip() + " "
            f.write(output.strip())
            f.write('\r\n')
            i += 1
            if i % 10000 == 0:
                j += 1
                w.set_description(u'已分词： %s万个句子'%j)
    f.close()    

结果如下：

![](img/exp7_img1.png)

### 3、gensim训练词向量
  gensim训练词向量分为三步，第一步获取sentences，第二部设置超参数，第三步模型保存。

- 获取sentences：sentences是已经分词过的字符串列表，其为一维数组，可直接读取`word.txt`文件。
- 设置超参数：word2vec模型的超参数如下所示：
<br>
(1) sentences: 我们要分析的语料，可以是一个列表，或者从文件中遍历读出。后面我们会有从文件读出的例子。
<br>
(2) vector_size: 词向量的维度，默认值是100。这个维度的取值一般与我们的语料的大小相关，如果是不大的语料，比如小于100M的文本语料，则使用默认值一般就可以了。如果是超大的语料，建议增大维度。
<br>
(3) window：即词向量上下文最大距离，这个参数在我们的算法原理篇中标记为c，window越大，则和某一词较远的词也会产生上下文关系。默认值为5。在实际使用中，可以根据实际的需求来动态调整这个window的大小。如果是小语料则这个值可以设的更小。对于一般的语料这个值推荐在[5,10]之间。
<br>
(4) sg: 即我们的word2vec两个模型的选择了。如果是0， 则是CBOW模型，是1则是Skip-Gram模型，默认是0即CBOW模型。
<br>
(5) hs: 即我们的word2vec两个解法的选择了，如果是0， 则是Negative Sampling，是1的话并且负采样个数negative大于0， 则是Hierarchical Softmax。默认是0即Negative Sampling。
<br>
(6) negative:即使用Negative Sampling时负采样的个数，默认是5。推荐在[3,10]之间。这个参数在我们的算法原理篇中标记为neg。
<br>
(7) cbow_mean: 仅用于CBOW在做投影的时候，为0，则算法中的xw为上下文的词向量之和，为1则为上下文的词向量的平均值。在我们的原理篇中，是按照词向量的平均值来描述的。个人比较喜欢用平均值来表示xw,默认值也是1,不推荐修改默认值。
<br>
(8) min_count:需要计算词向量的最小词频。这个值可以去掉一些很生僻的低频词，默认是5。如果是小语料，可以调低这个值。
<br>
(9) iter: 随机梯度下降法中迭代的最大次数，默认是5。对于大语料，可以增大这个值。
<br>
(10) alpha: 在随机梯度下降法中迭代的初始步长。算法原理篇中标记为η，默认是0.025。
<br>
(11) min_alpha: 由于算法支持在迭代的过程中逐渐减小步长，min_alpha给出了最小的迭代步长值。随机梯度下降中每轮的迭代步长可以由iter，alpha， min_alpha一起得出。这部分由于不是word2vec算法的核心内容，因此在原理篇我们没有提到。对于大语料，需要对alpha, min_alpha,iter一起调参，来选择合适的三个值。
<br>
- 模型保存：训练后的模型需要保存为文件格式，以便后续的读取和使用。模型保存只要一行代码：

### 4、训练模型

In [ ]:
from gensim.models.word2vec import Word2Vec 
sentences = []
file = 'wikizh/word.txt'
with open('./' + file,'r',encoding="utf-8") as f:
    for i in f.readlines():
        sentences.append(i)
model = Word2Vec(sentences,min_count=10,sg=0) # default value is 5
model.save('wikizh/wiki.zh.Model') 

### 5、读取模型

In [ ]:
import gensim
model = gensim.models.Word2Vec.load('wikizh/wiki.zh.Model')
#查看某个字词的向量：
print(model.wv['数'])
#查看与该词最接近的其他词汇及相似度：
print(model.wv.most_similar(['数']))
#查看两个词之间的相似度：
model.wv.similarity('数','值')

# 三、实验任务

1. 实现基于Word2vec和text8语料集的词向量表示。